In [10]:
import warnings
warnings.filterwarnings(action='ignore')
%config Computer.use_jedi = False
import requests
import folium
# !pip install pandas # 파이썬 데이터 처리 라이브러리
import pandas as pd
from pandas.io.json import json_normalize # 파이썬 딕셔너리 타입의 데이터를 판다스 데이터프레임으로 변환한다.

전국 또는 특정 지역의 스타벅스 매장 위치 정보를 가져와서 지도위에 표시하기

requests 모듈로 스타벅스 매장의 위치 정보를 가져와서 딕셔너리 타입으로 변환한다.  
json_normalize() 함수로 json 타입의 데이터가 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.  
folium 모듈을 사용해서 지도를 표시하고, 지도위에 스타벅스 매장 위치를 marker로 표시한다.

In [14]:
# 서울 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=HBSCEBENZ1'
request = requests.post(targetSite, data={
    'ins_lat': 37.563398, # 위도
    'ins_lng': 126.9863309, # 경도
    'p_sido_cd': '01', # 시도코드 
    'p_gugun_cd': '', # 구군코드 
    'in_biz_cd': '', # ?
    'iend': 1900, # 서버가 응답하는 최대 매장의 개수
    'set_date': '' #? 
})
starbucks = request.json()
#print(type(starbucks))
#print(starbucks)

In [18]:
# pandas.io.json 모듈의 json_normalize() 함수로 json 타입의 데이터가 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.
# json_nomalize(딕셔너리, '데이터프레임으로 변환할 데이터가 할당된 딕셔너리의 key')
starbucks_df = json_normalize(starbucks, 'list')
print(type(starbucks_df)) # <class 'pandas.core.frame.DataFrame'>
print(len(starbucks_df))
print(starbucks_df.columns)

<class 'pandas.core.frame.DataFrame'>
601
Index(['seq', 'sido_cd', 'sido_nm', 'gugun_cd', 'gugun_nm', 'code_order',
       'view_yn', 'store_num', 'sido', 'gugun',
       ...
       'p90', 'p01', 't05', 't30', 't36', 't27', 't29', 't43', 't48', 'z9999'],
      dtype='object', length=131)


In [19]:
starbucks_df

,seq,sido_cd,sido_nm,gugun_cd,gugun_nm,code_order,view_yn,store_num,sido,gugun,address,new_img_nm,p_pro_seq,p_view_yn,p_sido_cd,p_gugun_cd,p_store_nm,p_theme_cd,p_wireless_yn,p_smoking_yn,p_book_yn,p_music_yn,p_terrace_yn,p_table_yn,p_takeout_yn,p_parking_yn,p_dollar_assent,p_card_recharge,p_subway_yn,stb_store_file_renew,stb_store_theme_renew,stb_store_time_renew,stb_store_lsm,s_code,s_name,tel,fax,sido_code,sido_name,gugun_code,gugun_name,addr,park_info,new_state,theme_state,new_bool,search_text,ins_lat,ins_lng,in_distance,out_distance,all_search_cnt,addr_search_cnt,store_search_cnt,rowCount,store_nm,store_cd,s_biz_code,new_icon,set_user,...,in_biz_arrdata,in_scodes,in_scode_arr,in_scode_arrdata,disp,set_date,hlytag,hlytag_msg,vSal,istart,iend,open_dt,gold_card,ip_lat,ip_long,espresso,new_store,premiere_food,doro_address,cold_blew,my_siren_order_store_yn,whcroad_yn,skuNo,skuName,skuImgUrl,stock_count,store_area_name,store_area_code,is_open,gift_stock_yn,lat,lot,t20,t04,t03,t01,t12,t09,t06,t10,p10,p50,p20,p60,p30,p70,p40,p80,t22,t21,p90,p01,t05,t30,t36,t27,t29,t43,t48,z9999
0,0,None,None,None,None,None,None,None,None,None,None,None,0,None,,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1509,역삼아레나빌딩,1522-3232,02-568-3763,01,서울,0101,강남구,서울특별시 강남구 역삼동 721-13 아레나빌딩,None,None,Z9999@T05@T08@T16@T17@T20@T21@T30@@T52@T57@P80...,0,,,,0,8.55,-1,-1,-1,30,,0,3762,N,,...,None,0,None,None,None,None,None,None,,1,60,20190613,0,,,,,,서울특별시 강남구 언주로 425 (역삼동),,N,WHCROAD,,,,0,None,A01,None,None,37.501087,127.043069,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,None,None,None,None,None,None,None,None,None,None,None,0,None,,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1434,논현역사거리,1522-3232,02-3442-3673,01,서울,0101,강남구,서울특별시 강남구 논현동 142-2 정일빌딩,None,None,Z9999@T05@T08@T16@T17@T20@T21@T30@T52@T57@P70@...,0,,,,0,6.71,-1,-1,-1,30,,0,3672,N,,...,None,0,None,None,None,None,None,None,,1,60,20181123,0,,,,,,서울특별시 강남구 강남대로 538 (논현동),,N,,,,,0,None,A01,None,None,37.510178,127.022223,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,None,None,None,None,None,None,None,None,None,None,None,0,None,,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1595,신사역성일빌딩,1522-3232,02-547-3859,01,서울,0101,강남구,서울특별시 강남구 논현동 18-4 성일빌딩,None,None,Z9999@T05@T08@T16@T17@T20@T21@T30@T34@@T36@T52...,0,,,,0,6.28,-1,-1,-1,30,,0,3858,N,,...,None,0,None,None,None,None,None,None,,1,60,20191219,0,,,,,,서울특별시 강남구 강남대로 584 (논현동),,N,,,,,0,None,A01,None,None,37.5139309,127.0206057,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,None,None,None,None,None,None,None,None,None,None,None,0,None,,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1527,국기원사거리,1522-3232,02-568-3669,01,서울,0101,강남구,서울특별시 강남구 역삼동 648-22 동찬빌딩,None,None,Z9999@T05@T08@T16@T17@T20@T21@T30@T32@T34@@T52...,0,,,,0,8.14,-1,-1,-1,30,,0,3669,N,,...,None,0,None,None,None,None,None,None,,1,60,20190731,0,,,,,,서울특별시 강남구 테헤란로 125 (역삼동),,N,WHCROAD,,,,0,None,A01,None,None,37.499517,127.031495,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,None,None,None,None,None,None,None,None,None,None,None,0,None,,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1468,대치재경빌딩R,1522-3232,02-568-3705,01,서울,0101,강남구,서울특별시 강남구 대치동 599 대원빌딩,None,None,Z9999@T03@T05@T08@T16@T17@T20@T21@T22@T26@T29@...,0,,,,0,10.18,-1,-1,-1,30,,0,3703,N,,...,None,0,None,None,None,None,None,None,,1,60,20190214,0,,,,,,서울특별시 강남구 남부순환로 2947 (대치동),,N,WHCROAD,,,,0,None,A01,None,None,37.494668,127.062583,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [ ]:
# 데이터프레임으로 변환하니 서울에 있는 스타벅스 지점은 601개고 각 지점당 131개의 컬럼으로 정보를 표시하고 있다.
# 작업에 필요한 컬럼 몇 가지를 선택해서 지도에 마커를 표시할 때, 사용할 데이터가 저장된 데이터프레임을 만든다.
# s_name => 지점이름
# sido_code => 시도코드
# sido_name => 시도이름
# gugun_code => 구군코드
# gugun_name => 구군이름
# doro_address => 도로명주소
# lat => 위도
# lot => 경도

In [20]:
# 판다스 데이터프레임에서 특정 컬럼의 데이터를 얻어올 때 컬럼이 1개라면 컬럼이름만 적어도 되지만 얻어올
# 컬럼이 2개 이상일 경우 얻어올 컬럼들을 반드시 리스트로 만들어서 얻어와야 한다.
starbucks_df_map = starbucks_df[
    [
        's_name',
        'sido_code',
        'sido_name',
        'gugun_code',
        'gugun_name',
        'doro_address',
        'lat',
        'lot'
    ]
]
starbucks_df_map

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
0,역삼아레나빌딩,01,서울,0101,강남구,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
1,논현역사거리,01,서울,0101,강남구,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
2,신사역성일빌딩,01,서울,0101,강남구,서울특별시 강남구 강남대로 584 (논현동),37.5139309,127.0206057
3,국기원사거리,01,서울,0101,강남구,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
4,대치재경빌딩R,01,서울,0101,강남구,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583
...,...,...,...,...,...,...,...,...
596,사가정역,01,서울,0125,중랑구,서울특별시 중랑구 면목로 310,37.579594,127.087966
597,상봉역,01,서울,0125,중랑구,서울특별시 중랑구 망우로 307 (상봉동),37.59689,127.08647
598,묵동,01,서울,0125,중랑구,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633
599,양원역,01,서울,0125,중랑구,서울특별시 중랑구 양원역로10길 3 (망우동),37.6066536267232,127.106359790053


In [21]:
# dtype 속성으로 데이터프레임을 구성하는 컬럼들의 자료형을 확인할 수 있다.
# 판다스는 문자열 데이터 타입을 object라 부른다.
starbucks_df_map.dtypes

s_name          object
sido_code       object
sido_name       object
gugun_code      object
gugun_name      object
doro_address    object
lat             object
lot             object
dtype: object

In [22]:
# info() 메소드를 사용해도 데이터프레임을 구성하는 컬럼들의 자료형을 확인할 수 있다.
starbucks_df_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601 entries, 0 to 600
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   s_name        601 non-null    object
 1   sido_code     601 non-null    object
 2   sido_name     601 non-null    object
 3   gugun_code    601 non-null    object
 4   gugun_name    601 non-null    object
 5   doro_address  601 non-null    object
 6   lat           601 non-null    object
 7   lot           601 non-null    object
dtypes: object(8)
memory usage: 37.7+ KB


In [23]:
# astype() 메소드로 위도, 경도 데이터 타입을 object에서 float로 변환한다.
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)
starbucks_df_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601 entries, 0 to 600
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   s_name        601 non-null    object 
 1   sido_code     601 non-null    object 
 2   sido_name     601 non-null    object 
 3   gugun_code    601 non-null    object 
 4   gugun_name    601 non-null    object 
 5   doro_address  601 non-null    object 
 6   lat           601 non-null    float64
 7   lot           601 non-null    float64
dtypes: float64(2), object(6)
memory usage: 37.7+ KB


In [24]:
# 종각점
starbucks_df_map[starbucks_df_map['s_name'] == '종각']

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
443,종각,01,서울,0118,종로구,서울특별시 종로구 종로 64 (종로2가),37.569918,126.984528


In [25]:
# 종로 관철점
starbucks_df_map[starbucks_df_map['s_name'] == '종로관철']

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
445,종로관철,01,서울,0118,종로구,"서울특별시 종로구 종로12길 21, 2층 (관철동)",37.569058,126.986013


In [26]:
# 종로구청점
starbucks_df_map[starbucks_df_map['s_name'] == '종로구청']

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
430,종로구청,01,서울,0118,종로구,서울특별시 종로구 삼봉로 71 (수송동),37.572662,126.981138


In [29]:
starbucksMap = folium.Map(location=[37.56961146, 126.9840133], zoom_start=17)
popup = folium.Popup('종각점', max_width=300)
folium.Marker(location=[37.569918, 126.984528], popup=popup).add_to(starbucksMap)
popup = folium.Popup('종로관철점', max_width=300)
folium.Marker(location=[37.569058, 126.986013], popup=popup).add_to(starbucksMap)
popup = folium.Popup('종로구청점', max_width=300)
folium.Marker(location=[37.572662, 126.981138], popup=popup).add_to(starbucksMap)
starbucksMap

In [33]:
# 서울특별시에 위치한 스타벅스 전 지점의 위치를 지도위에 표시한다.
# 지도를 작성할 때 지도의 중심으로 설정할 위치가 애매할 경우 지도위에 표시할 모든 스타벅스 지점의 위도와 경도의 평균을 중심으로 하는 지도를 만들어 사용하면 된다.
# 데이터프레임에 저장된 데이터의 평균은 mean() 메소드를 사용해서 계산한다.
starbucksMap = folium.Map(location=[starbucks_df_map['lat'].mean(), starbucks_df_map['lot'].mean()], zoom_start=17)
# iterrows() 메소드는 데이터프레임에 저장된 데이터의 인덱스와 데이터를 리턴한다.
for index, data in starbucks_df_map.iterrows():
    #print(index, data)
    popup = folium.Popup('{}. {} - {}'.format(index + 1, data['s_name'], data['doro_address']), max_width=300)
    folium.Marker(location=[data['lat'], data['lot']], popup=popup).add_to(starbucksMap)

starbucksMap

In [35]:
# 서울 종로구 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=HBSCEBENZ1'
request = requests.post(targetSite, data={
    'ins_lat': 37.563398, # 위도
    'ins_lng': 126.9863309, # 경도
    'p_sido_cd': '', # 시도코드 
    'p_gugun_cd': '0118', # 구군코드 
    'in_biz_cd': '', # ?
    'iend': 1900, # 서버가 응답하는 최대 매장의 개수
    'set_date': '' #? 
})
starbucks = request.json()
starbucks_df = json_normalize(starbucks, 'list')
starbucks_df_map = starbucks_df[['s_name','sido_code','sido_name','gugun_code','gugun_name', 'doro_address','lat','lot']]
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)

starbucksMap = folium.Map(location=[starbucks_df_map['lat'].mean(), starbucks_df_map['lot'].mean()], zoom_start=14)
for index, data in starbucks_df_map.iterrows():
    popup = folium.Popup('{}. {} - {}'.format(index + 1, data['s_name'], data['doro_address']), max_width=300)
    folium.Marker(location=[data['lat'], data['lot']], popup=popup).add_to(starbucksMap)

starbucksMap